<a target="_blank" href="https://colab.research.google.com/github/wakusoftware/curso-ml-espanol/blob/master/C1%20-%20Aprendizaje%20Supervisado/W2/C1_W2_Lab01_Numpy_Vectorizacion.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Setup para Colab
Si estás corriendo este Notebook en Google Colab corre la celda de abajo, de lo contrario ignórala.

In [ ]:
!git clone https://github.com/wakusoftware/curso-ml-espanol.git

%cd curso-ml-espanol/C1 - Aprendizaje Supervisado/W2/

!cp lab_utils_common.py /content/

!cp lab_utils_multi.py /content/

!cp deeplearning.mplstyle /content/

!cp -r data /content/

!cp -r images /content/

%cd /content/

!rm -rf curso-ml-espanol/

# Laboratorio: Regresión Lineal Múltiple

En este laboratorio, extenderás las estructuras de datos y las rutinas desarrolladas anteriormente para soportar múltiples características. Varias rutinas se actualizan haciendo que el laboratorio parezca extenso, pero solo realiza ajustes menores a las rutinas anteriores, lo que facilita su revisión.
# Contenido
- [&nbsp;&nbsp;1.1 Objetivos](#toc_15456_1.1)
- [&nbsp;&nbsp;1.2 Herramientas](#toc_15456_1.2)
- [&nbsp;&nbsp;1.3 Notación](#toc_15456_1.3)
- [2 Planteamiento del Problema](#toc_15456_2)
- [&nbsp;&nbsp;2.1 Matriz X que contiene nuestros ejemplos](#toc_15456_2.1)
- [&nbsp;&nbsp;2.2 Vector de parámetros w, b](#toc_15456_2.2)
- [3 Predicción del Modelo con Múltiples Variables](#toc_15456_3)
- [&nbsp;&nbsp;3.1 Predicción individual elemento por elemento](#toc_15456_3.1)
- [&nbsp;&nbsp;3.2 Predicción individual, vector](#toc_15456_3.2)
- [4 Cálculo del Costo con Múltiples Variables](#toc_15456_4)
- [5 Descenso del Gradiente con Múltiples Variables](#toc_15456_5)
- [&nbsp;&nbsp;5.1 Calcular el Gradiente con Múltiples Variables](#toc_15456_5.1)
- [&nbsp;&nbsp;5.2 Descenso del Gradiente con Múltiples Variables](#toc_15456_5.2)
- [6 Felicitaciones](#toc_15456_6)

<a name="toc_15456_1.1"></a>
## 1.1 Objetivos
- Extender nuestras rutinas del modelo de regresión para soportar múltiples características
    - Extender las estructuras de datos para soportar múltiples características
    - Reescribir las rutinas de predicción, costo y gradiente para soportar múltiples características
    - Utilizar NumPy `np.dot` para vectorizar sus implementaciones por velocidad y simplicidad

<a name="toc_15456_1.2"></a>
## 1.2 Herramientas
En este laboratorio, haremos uso de: 
- NumPy, una biblioteca popular para computación científica
- Matplotlib, una biblioteca popular para la visualización de datos

In [ ]:
import copy, math
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('./deeplearning.mplstyle')
np.set_printoptions(precision=2)  # precisión reducida en la visualización de arrays de numpy

<a name="toc_15456_1.3"></a>
## 1.3 Notación
Aquí tienes un resumen de algunas de las notaciones que encontrarás, actualizadas para múltiples características.

|Notación General <img width=70/> | Descripción<img width=350/>| Python (si aplica) |
|: ------------|: ------------------------------------------------------------|:----------------|
| $a$ | escalar, no en negrita                                                      ||
| $\mathbf{a}$ | vector, en negrita                                                 ||
| $\mathbf{A}$ | matriz, en negrita y mayúscula                                         ||
| **Regresión** |         |    |     |
|  $\mathbf{X}$ | matriz de ejemplos de entrenamiento                  | `X_train` |   
|  $\mathbf{y}$  | objetivos de los ejemplos de entrenamiento                | `y_train` 
|  $\mathbf{x}^{(i)}$, $y^{(i)}$ | Ejemplo de entrenamiento $i_{th}$ | `X[i]`, `y[i]`|
| m | número de ejemplos de entrenamiento | `m`|
| n | número de características en cada ejemplo | `n`|
|  $\mathbf{w}$  |  parámetro: peso                       | `w`    |
|  $b$           |  parámetro: sesgo                                           | `b`    |     
| $f_{\mathbf{w},b}(\mathbf{x}^{(i)})$ | El resultado de la evaluación del modelo en $\mathbf{x}^{(i)}$ parametrizado por $\mathbf{w},b$: $f_{\mathbf{w},b}(\mathbf{x}^{(i)}) = \mathbf{w} \cdot \mathbf{x}^{(i)}+b$  | `f_wb` |

<a name="toc_15456_2"></a>
# 2 Planteamiento del Problema

Utilizarás el ejemplo motivador de predicción de precios de viviendas. El conjunto de datos de entrenamiento contiene tres ejemplos con cuatro características (tamaño, habitaciones, pisos y edad) mostrados en la tabla a continuación. Ten en cuenta que, a diferencia de los laboratorios anteriores, el tamaño está en pies cuadrados en lugar de 1000 pies cuadrados. ¡Esto causa un problema, que resolverás en el próximo laboratorio!

| Tamaño (pies cuadrados) | Número de habitaciones  | Número de pisos | Edad de la vivienda | Precio (miles de dólares)  |   
| ----------------| ------------------- |----------------- |--------------|-------------- |  
| 2104            | 5                   | 1                | 45           | 460           |  
| 1416            | 3                   | 2                | 40           | 232           |  
| 852             | 2                   | 1                | 35           | 178           |  

Construirás un modelo de regresión lineal utilizando estos valores para que luego puedas predecir el precio de otras casas. Por ejemplo, una casa de 1200 pies cuadrados, 3 habitaciones, 1 piso, 40 años de antigüedad.

Por favor, ejecuta la siguiente celda de código para crear tus variables `X_train` y `y_train`.

In [ ]:
X_train = np.array([[2104, 5, 1, 45], [1416, 3, 2, 40], [852, 2, 1, 35]])
y_train = np.array([460, 232, 178])

<a name="toc_15456_2.1"></a>
## 2.1 Matriz X que contiene nuestros ejemplos
Similar a la tabla anterior, los ejemplos se almacenan en una matriz de NumPy `X_train`. Cada fila de la matriz representa un ejemplo. Cuando tienes $m$ ejemplos de entrenamiento (en nuestro ejemplo $m$ es tres), y hay $n$ características (cuatro en nuestro ejemplo), $\mathbf{X}$ es una matriz con dimensiones ($m$, $n$) (m filas, n columnas).

$$\mathbf{X} = 
\begin{pmatrix}
 x^{(0)}_0 & x^{(0)}_1 & \cdots & x^{(0)}_{n-1} \\ 
 x^{(1)}_0 & x^{(1)}_1 & \cdots & x^{(1)}_{n-1} \\
 \cdots \\
 x^{(m-1)}_0 & x^{(m-1)}_1 & \cdots & x^{(m-1)}_{n-1} 
\end{pmatrix}
$$
notación:
- $\mathbf{x}^{(i)}$ es el vector que contiene el ejemplo i. $\mathbf{x}^{(i)}$ $ = (x^{(i)}_0, x^{(i)}_1, \cdots, x^{(i)}_{n-1})$
- $x^{(i)}_j$ es el elemento j en el ejemplo i. El superíndice entre paréntesis indica el número de ejemplo mientras que el subíndice representa un elemento.

Muestra los datos de entrada.

In [ ]:
# los datos están almacenados en un array/matriz de numpy
print(f"Forma de X: {X_train.shape}, Tipo de X:{type(X_train)})")
print(X_train)
print(f"Forma de y: {y_train.shape}, Tipo de y:{type(y_train)})")
print(y_train)

<a name="toc_15456_2.2"></a>
## 2.2 Vector de parámetros w, b

* $\mathbf{w}$ es un vector con $n$ elementos.
  - Cada elemento contiene el parámetro asociado con una característica.
  - en nuestro conjunto de datos, n es 4.
  - conceptualmente, lo representamos como un vector columna

$$\mathbf{w} = \begin{pmatrix}
w_0 \\ 
w_1 \\
\cdots\\
w_{n-1}
\end{pmatrix}
$$
* $b$ es un parámetro escalar.  

Para la demostración, $\mathbf{w}$ y $b$ se cargarán con algunos valores iniciales seleccionados que están cerca del óptimo. $\mathbf{w}$ es un vector NumPy de 1-D.

In [ ]:
b_init = 785.1811367994083
w_init = np.array([0.39133535, 18.75376741, -53.36032453, -26.42131618])
print(f"forma de w_init: {w_init.shape}, tipo de b_init: {type(b_init)}")

<a name="toc_15456_3"></a>
# 3 Predicción del Modelo con Múltiples Variables
La predicción del modelo con múltiples variables se da por el modelo lineal:

$$ f_{\mathbf{w},b}(\mathbf{x}) =  w_0x_0 + w_1x_1 +... + w_{n-1}x_{n-1} + b \tag{1}$$
o en notación vectorial:
$$ f_{\mathbf{w},b}(\mathbf{x}) = \mathbf{w} \cdot \mathbf{x} + b  \tag{2} $$ 
donde $\cdot$ es un `producto punto` vectorial

Para demostrar el producto punto, implementaremos la predicción usando (1) y (2).

<a name="toc_15456_3.1"></a>
## 3.1 Predicción individual elemento por elemento
Nuestra predicción anterior multiplicaba un valor de característica por un parámetro y añadía un parámetro de sesgo. Una extensión directa de nuestra implementación anterior de la predicción a múltiples características sería implementar (1) anterior usando un bucle sobre cada elemento, realizando la multiplicación con su parámetro y luego añadiendo el parámetro de sesgo al final.

In [ ]:
def predict_single_loop(x, w, b): 
    """
    predicción única usando regresión lineal
    
    Args:
      x (ndarray): Forma (n,) ejemplo con múltiples características
      w (ndarray): Forma (n,) parámetros del modelo    
      b (escalar):  parámetro del modelo     
      
    Returns:
      p (escalar):  predicción
    """
    n = x.shape[0]
    p = 0
    for i in range(n):
        p_i = x[i] * w[i]  
        p = p + p_i         
    p = p + b                
    return p


In [ ]:
# obtener una fila de nuestros datos de entrenamiento
x_vec = X_train[0,:]
print(f"forma de x_vec {x_vec.shape}, valor de x_vec: {x_vec}")

# hacer una predicción
f_wb = predict_single_loop(x_vec, w_init, b_init)
print(f"forma de f_wb {f_wb.shape}, predicción: {f_wb}")


Nota la forma de `x_vec`. Es un vector NumPy 1-D con 4 elementos, (4,). El resultado, `f_wb`, es un escalar.

<a name="toc_15456_3.2"></a>
## 3.2 Predicción individual, vector

Observando que la ecuación (1) anterior puede implementarse usando el producto punto como en (2) arriba. Podemos hacer uso de operaciones vectoriales para acelerar las predicciones.

Recuerda del laboratorio de Python/NumPy que NumPy `np.dot()`[[enlace](https://numpy.org/doc/stable/reference/generated/numpy.dot.html)] puede usarse para realizar un producto punto vectorial.

In [ ]:
def predict(x, w, b): 
    """
    predicción única usando regresión lineal
    Args:
      x (ndarray): Forma (n,) ejemplo con múltiples características
      w (ndarray): Forma (n,) parámetros del modelo   
      b (escalar): parámetro del modelo 
      
    Returns:
      p (escalar):  predicción
    """
    p = np.dot(x, w) + b     
    return p    


In [ ]:
# obtener una fila de nuestros datos de entrenamiento
x_vec = X_train[0,:]
print(f"forma de x_vec {x_vec.shape}, valor de x_vec: {x_vec}")

# hacer una predicción
f_wb = predict(x_vec, w_init, b_init)
print(f"forma de f_wb {f_wb.shape}, predicción: {f_wb}")


Los resultados y las formas son los mismos que en la versión anterior que utilizaba bucles. En adelante, se utilizará `np.dot` para estas operaciones. La predicción es ahora una sola declaración. La mayoría de las rutinas la implementarán directamente en lugar de llamar a una rutina de predicción separada.

<a name="toc_15456_4"></a>
# 4 Calcular Costo con Múltiples Variables
La ecuación para la función de costo con múltiples variables $J(\mathbf{w},b)$ es:
$$J(\mathbf{w},b) = \frac{1}{2m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)})^2 \tag{3}$$ 
donde:
$$ f_{\mathbf{w},b}(\mathbf{x}^{(i)}) = \mathbf{w} \cdot \mathbf{x}^{(i)} + b  \tag{4} $$ 

A diferencia de laboratorios anteriores, $\mathbf{w}$ y $\mathbf{x}^{(i)}$ son vectores en lugar de escalares, lo que permite soportar múltiples características.

A continuación se muestra una implementación de las ecuaciones (3) y (4). Ten en cuenta que esto utiliza un *patrón estándar para este curso* donde se usa un bucle for sobre todos los ejemplos `m`.

In [ ]:
def compute_cost(X, y, w, b): 
    """
    calcular costo
    Args:
      X (ndarray (m,n)): Datos, m ejemplos con n características
      y (ndarray (m,)) : valores objetivo
      w (ndarray (n,)) : parámetros del modelo  
      b (escalar)      : parámetro del modelo
      
    Returns:
      cost (escalar): costo
    """
    m = X.shape[0]
    cost = 0.0
    for i in range(m):                                
        f_wb_i = np.dot(X[i], w) + b           #(n,)(n,) = escalar (ver np.dot)
        cost = cost + (f_wb_i - y[i])**2       #escalar
    cost = cost / (2 * m)                      #escalar    
    return cost


In [ ]:
# Calcular y mostrar el costo usando nuestros parámetros óptimos preseleccionados.
cost = compute_cost(X_train, y_train, w_init, b_init)
print(f'Costo con w óptimo: {cost}')

**Resultado Esperado**: Costo con w óptimo: 1.5578904045996674e-12

<a name="toc_15456_5"></a>
# 5 Descenso del Gradiente con Múltiples Variables
Descenso del gradiente para múltiples variables:

$$\begin{align*} \text{repetir}&\text{ hasta converger:} \; \lbrace \newline\;
& w_j = w_j -  \alpha \frac{\partial J(\mathbf{w},b)}{\partial w_j} \tag{5}  \; & \text{para j = 0..n-1}\newline
&b\ \ = b -  \alpha \frac{\partial J(\mathbf{w},b)}{\partial b}  \newline \rbrace
\end{align*}$$

donde, n es el número de características, los parámetros $w_j$,  $b$, se actualizan simultáneamente y donde  

$$
\begin{align}
\frac{\partial J(\mathbf{w},b)}{\partial w_j}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)})x_{j}^{(i)} \tag{6}  \\
\frac{\partial J(\mathbf{w},b)}{\partial b}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)}) \tag{7}
\end{align}
$$
* m es el número de ejemplos de entrenamiento en el conjunto de datos

    
*  $f_{\mathbf{w},b}(\mathbf{x}^{(i)})$ es la predicción del modelo, mientras que $y^{(i)}$ es el valor objetivo

<a name="toc_15456_5.1"></a>
## 5.1 Calcular el Gradiente con Múltiples Variables
A continuación se presenta una implementación para calcular las ecuaciones (6) y (7). Hay muchas maneras de implementar esto. En esta versión, hay un
- bucle externo sobre todos los ejemplos m.
    - $\frac{\partial J(\mathbf{w},b)}{\partial b}$ para el ejemplo se puede calcular directamente y acumular
    - en un segundo bucle sobre todas las características n:
        - $\frac{\partial J(\mathbf{w},b)}{\partial w_j}$ se calcula para cada $w_j$.

In [ ]:
def compute_gradient(X, y, w, b): 
    """
    Calcula el gradiente para la regresión lineal 
    Args:
      X (ndarray (m,n)): Datos, m ejemplos con n características
      y (ndarray (m,)) : valores objetivo
      w (ndarray (n,)) : parámetros del modelo  
      b (escalar)      : parámetro del modelo
      
    Returns:
      dj_dw (ndarray (n,)): El gradiente del costo respecto a los parámetros w. 
      dj_db (escalar):       El gradiente del costo respecto al parámetro b. 
    """
    m, n = X.shape           #(número de ejemplos, número de características)
    dj_dw = np.zeros((n,))
    dj_db = 0.

    for i in range(m):                             
        err = (np.dot(X[i], w) + b) - y[i]   
        for j in range(n):                         
            dj_dw[j] = dj_dw[j] + err * X[i, j]    
        dj_db = dj_db + err                        
    dj_dw = dj_dw / m                                
    dj_db = dj_db / m                                
        
    return dj_db, dj_dw


In [ ]:
# Calcular y mostrar el gradiente
tmp_dj_db, tmp_dj_dw = compute_gradient(X_train, y_train, w_init, b_init)
print(f'dj_db con w,b iniciales: {tmp_dj_db}')
print(f'dj_dw con w,b iniciales: \n {tmp_dj_dw}')

**Resultado Esperado**:   
dj_db con w,b iniciales: -1.6739251122999121e-06  
dj_dw con w,b iniciales:   
 [-2.73e-03 -6.27e-06 -2.22e-06 -6.92e-05]  

<a name="toc_15456_5.2"></a>
## 5.2 Descenso del Gradiente con Múltiples Variables
La rutina a continuación implementa la ecuación (5) mencionada anteriormente.

In [ ]:
def gradient_descent(X, y, w_in, b_in, cost_function, gradient_function, alpha, num_iters): 
    """
    Realiza descenso de gradiente por lotes para aprender theta. Actualiza theta tomando 
    num_iters pasos de gradiente con la tasa de aprendizaje alpha
    
    Args:
      X (ndarray (m,n))   : Datos, m ejemplos con n características
      y (ndarray (m,))    : valores objetivo
      w_in (ndarray (n,)) : parámetros iniciales del modelo  
      b_in (escalar)      : parámetro inicial del modelo
      cost_function       : función para calcular el costo
      gradient_function   : función para calcular el gradiente
      alpha (float)       : Tasa de aprendizaje
      num_iters (int)     : número de iteraciones para ejecutar el descenso de gradiente
      
    Returns:
      w (ndarray (n,)) : Valores actualizados de los parámetros 
      b (escalar)      : Valor actualizado del parámetro 
      """
    
    # Un arreglo para almacenar el costo J y los valores de w en cada iteración principalmente para graficar luego
    J_history = []
    w = copy.deepcopy(w_in)  #evitar modificar el w global dentro de la función
    b = b_in
    
    for i in range(num_iters):

        # Calcular el gradiente y actualizar los parámetros
        dj_db,dj_dw = gradient_function(X, y, w, b)   ##None

        # Actualizar parámetros usando w, b, alpha y gradiente
        w = w - alpha * dj_dw               ##None
        b = b - alpha * dj_db               ##None
      
        # Guardar el costo J en cada iteración
        if i<100000:      # evitar agotamiento de recursos 
            J_history.append( cost_function(X, y, w, b))

        # Imprimir el costo cada cierto intervalo, 10 veces o tantas iteraciones si son < 10
        if i% math.ceil(num_iters / 10) == 0:
            print(f"Iteración {i:4d}: Costo {J_history[-1]:8.2f}   ")
        
    return w, b, J_history #devolver el w, b y el historial de J finales para graficar


En la siguiente celda probarás la implementación.

In [ ]:
# inicializar parámetros
initial_w = np.zeros_like(w_init)
initial_b = 0.
# configuraciones de descenso de gradiente
iterations = 1000
alpha = 5.0e-7
# ejecutar descenso de gradiente 
w_final, b_final, J_hist = gradient_descent(X_train, y_train, initial_w, initial_b,
                                                    compute_cost, compute_gradient, 
                                                    alpha, iterations)
print(f"b, w encontrados por descenso de gradiente: {b_final:0.2f},{w_final} ")
m,_ = X_train.shape
for i in range(m):
    print(f"predicción: {np.dot(X_train[i], w_final) + b_final:0.2f}, valor objetivo: {y_train[i]}")


**Resultado Esperado**:    
b, w encontrados por descenso de gradiente: -0.00,[ 0.2   0.   -0.01 -0.07]   
predicción: 426.19, valor objetivo: 460  
predicción: 286.17, valor objetivo: 232  
predicción: 171.47, valor objetivo: 178  

In [ ]:
# graficar costo versus iteración
fig, (ax1, ax2) = plt.subplots(1, 2, constrained_layout=True, figsize=(12, 4))
ax1.plot(J_hist)
ax2.plot(100 + np.arange(len(J_hist[100:])), J_hist[100:])
ax1.set_title("Costo vs. iteración");  ax2.set_title("Costo vs. iteración (final)")
ax1.set_ylabel('Costo')             ;  ax2.set_ylabel('Costo') 
ax1.set_xlabel('paso de iteración')   ;  ax2.set_xlabel('paso de iteración') 
plt.show()


*¡Estos resultados no son inspiradores*! El costo aún está disminuyendo y nuestras predicciones no son muy precisas. El próximo laboratorio explorará cómo mejorar esto.

<a name="toc_15456_6"></a>
# ¡6 Felicitaciones!
En este laboratorio:
- Desarrollaste de nuevo las rutinas para la regresión lineal, ahora con múltiples variables.
- Utilizaste NumPy `np.dot` para vectorizar las implementaciones.